In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

In [2]:
import pandas as pd
import numpy as np

In [4]:
import requests
import random

In [ ]:
df = pd.read_csv("news_df.csv")

In [ ]:
df

In [ ]:
df_nan = df[df.full_text.isnull() == True]

In [ ]:
opts = webdriver.ChromeOptions()
opts.headless =True

In [ ]:
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [ ]:
search_url = "https://www.proquest.com/washingtonpost"
driver.get(search_url)


In [ ]:
driver.find_element(By.XPATH, '//*[@id="product-level-nav"]/li[2]').click()

In [ ]:
dataset = []
num_articles_scraped = 0
pattern_date = '[0-9]{1,2}\s[A-Za-z]{3,5}\s[0-9]{4}'

In [ ]:
def extract_articles(url, keys, dataset):
    time.sleep(5)
    driver_temp = webdriver.Chrome(options = opts)
    driver_temp.implicitly_wait(10)

    articles = driver.find_elements(By.CLASS_NAME, 'resultItem')
    driver_temp.get(url)
    
    if len(articles) == 0:
        print("Could not find the article with the title:\n" + keys + "\n")
        return None
    
    for article in articles:

        title = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("title")
        if keys != title:
            break
        date_text = article.find_element(By.CLASS_NAME,'newspaperArticle').text
        date = re.search(pattern_date, date_text).group()

        link = article.find_element(By.XPATH, './/*[@id="citationDocTitleLink"]').get_attribute("href")


        driver_temp.execute_script("window.open('');")
        driver_temp.switch_to.window(driver_temp.window_handles[1])
        driver_temp.get(link)

        try:
            WebDriverWait(driver_temp, 5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="fullTextZone"]/text'))
            )
            full_text = driver_temp.find_element(By.XPATH, '//*[@id="fullTextZone"]/text').text
        except:
            full_text = np.nan

        try:
            abstract = driver_temp.find_element(By.CLASS_NAME,'abstract').text
        except NoSuchElementException:
            abstract = np.nan


        article_info = {'date': date, 'title': title, "abstract": abstract, 'full_text': full_text}
        dataset.append(article_info)
        global num_articles_scraped
        num_articles_scraped += 1
        print(f"Number of scraped articles: {num_articles_scraped}")
        driver_temp.close()
        driver_temp.switch_to.window(driver_temp.window_handles[0])
        time.sleep(random.uniform(5, 10))

    driver_temp.quit()


In [ ]:
months = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr":4,"May":5,"June":6,
          "July":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}


for index in range(len(df_nan)):
    s = df_nan.date.iloc[index].split(" ")
    day = int(s[0])
    month = months[s[1]]
    year = s[2]
    
    driver.find_element(By.XPATH,'//*[@id="select_multiDateRange"]/option[7]').click()
    driver.find_element(By.XPATH,f'//*[@id="month2"]/option[{month + 1}]').click()
    driver.find_element(By.XPATH,f'//*[@id="day2"]/option[{day + 1}]').click()
    driver.find_element(By.XPATH,'//*[@id="year2"]').clear()
    driver.find_element(By.XPATH,'//*[@id="year2"]').send_keys(year)
    
    driver.find_element(By.XPATH,'//*[@id="SourceType_Newspapers"]').click()
    driver.find_element(By.XPATH,'//*[@id="Language_ENG"]').click()
    
    keys = df_nan.title.iloc[index]
    driver.find_element(By.XPATH, '//*[@id="queryTermField"]').clear()
    driver.find_element(By.XPATH, '//*[@id="queryTermField"]').send_keys(keys)
    driver.find_element(By.XPATH, '//*[@id="searchToResultPage"]').click()
    
    extract_articles(driver.current_url, keys ,dataset)
    driver.back()
    time.sleep(5)
    
df_nan_scraped = pd.DataFrame(dataset)
driver.quit()
    

In [ ]:
#df_nan_scraped.to_csv("news_data_scraped_nan.csv", index = False)

In [ ]:
df_nan_temp = df[df.full_text.isnull() == True]

In [ ]:
index_list = df_nan_temp.index

In [ ]:
for index in index_list:
    date = df.loc[index, 'date']
    title = df.loc[index, 'title']
    
    temp_index = (df_nan_scraped.title == title) & (df_nan_scraped.date == date)
    
    full_text = df_nan_scraped.full_text[temp_index]
    abstract = df_nan_scraped.abstract[temp_index]
    
    if len(abstract) != 0:
        df.loc[index, 'abstract'] = abstract.iloc[0]
        
    if len(full_text) != 0:
         df.loc[index, 'full_text'] = full_text.iloc[0]


In [ ]:
df_nan_temp = df[df.full_text.isnull() == True]

In [ ]:
df_nan_temp

In [ ]:
df.drop(df_nan_temp.index, axis = 0, inplace = True)

In [ ]:
#df.to_csv("cleaned_news_data.csv", index = False)

In [88]:
df_cleaned = pd.read_csv("cleaned_news_data.csv")

In [90]:
df_cleaned.drop_duplicates(inplace = True, ignore_index = True)

In [91]:
df_cleaned

,date,title,abstract,full_text
0,01 Jan 2000,The Clintons Host A Historic Fete; A Repast fo...,"Muhammad Ali, Elizabeth Taylor, Sophia Loren, ...",It was a once-in-a lifetime invitation--greeti...
1,01 Jan 2000,Bonds' Bad Year Closes With Whimper: [FINAL Ed...,"""Treasuries are still falling this morning, ta...",While tech stocks boomed and most major equity...
2,01 Jan 2000,Technology-Led Market Keeps Up Momentum: [FINA...,The year's star was the tech-laden Nasdaq comp...,"Stocks ended the year on an up note, capping t..."
3,01 Jan 2000,100 Lives; This Is Who We Were.: [FINAL Edition],My name is Larissa Kosmos. I'm 28. I'm blessed...,My name is Larissa Kosmos. I'm 28. I'm blessed...
4,02 Jan 2000,D.C. Homicides Lowest Since 1987; Killings in ...,Correction: An article Sunday incorrectly repo...,Correction: An article Sunday incorrectly repo...
...,...,...,...,...
17192,26 Dec 2021,"2020 was the craziest, creepiest year - until ...",NaN,"This is the craziest, creepiest year in both t..."
17193,26 Dec 2021,"Portraits from inside the messy, overheating a...",NaN,The U.S. economy this year was strong and scar...
17194,26 Dec 2021,Economy of 2021: Wild ride leaves few safe gue...,NaN,"Now, the climate is shifting again. In 2022, t..."
17195,30 Dec 2021,Continuing inflation set to be a top issue in ...,NaN,Little relief expected as prices rise at faste...


In [92]:
keys_F = ['federal reserve', ' fed ']
keys_M = ['money supply', 'open market operations',
'quantitative easing', 'monetary policy', 'fed funds rate', 'overnight lending rate',
' Bernanke ', ' Volker ', ' Greenspan ', ' Yellen ', ' Powell ', 'central bank', 'interest rates', 'fed chairman',
'fed chair', 'lender of last resort', 'discount window', 'European Central Bank',' ECB ', 'Bank of England',
'Bank of Japan',' BOJ ', 'Bank of China', 'Bundesbank', 'Bank of France', 'Bank of Italy']

In [93]:
dataset = []

In [94]:
for index in range(len(df_cleaned)):
    
    full_text = df_cleaned.full_text[index].lower()
    condition_satisfied = False
    
    for key_F in keys_F:
        
        for key_M in keys_M:
            
            condition = (key_F in (" " + full_text)) and (key_M.lower() in (" " + full_text))
            
            if condition is True:
                article_info = {'date': df_cleaned.date[index], 'title': df_cleaned.title[index],
                                "abstract": df_cleaned.abstract[index], 'full_text': df_cleaned.full_text[index]}
                dataset.append(article_info)
                
                condition_satisfied = True
                break
        
        if condition_satisfied:
            break
            

In [95]:
data = pd.DataFrame(dataset)

In [96]:
data

,date,title,abstract,full_text
0,01 Jan 2000,Bonds' Bad Year Closes With Whimper: [FINAL Ed...,"""Treasuries are still falling this morning, ta...",While tech stocks boomed and most major equity...
1,01 Jan 2000,Technology-Led Market Keeps Up Momentum: [FINA...,The year's star was the tech-laden Nasdaq comp...,"Stocks ended the year on an up note, capping t..."
2,01 Jan 2000,100 Lives; This Is Who We Were.: [FINAL Edition],My name is Larissa Kosmos. I'm 28. I'm blessed...,My name is Larissa Kosmos. I'm 28. I'm blessed...
3,02 Jan 2000,"After the Boom, More of the Same?: [FINAL Edit...",Part of the growth is due to the constant incr...,Next month the nation's extraordinary economic...
4,02 Jan 2000,Basis Points: [FINAL Edition],John M. Berry notes that the Treasury will sel...,Bond yields generally closed the week and the ...
...,...,...,...,...
9399,20 Dec 2021,Fill the vacancies,NaN,President Biden has yet to name anyone to fill...
9400,22 Dec 2021,Mr. Summers's courage,NaN,Lawrence H. Summers continues to make the case...
9401,22 Dec 2021,Just accept that Biden is not the new FDR,NaN,Something called Build Back Better may eventua...
9402,26 Dec 2021,"2020 was the craziest, creepiest year - until ...",NaN,"This is the craziest, creepiest year in both t..."


In [98]:
#data.to_csv("news_data.csv", index = False)